# 💉 2.3 依賴注入 (Dependency Injection)

依賴注入是 FastAPI 最強大的功能之一，讓程式碼更模組化、可測試！

---

## 🎯 學習目標

- ✅ 理解什麼是依賴注入
- ✅ 使用 `Depends` 建立依賴
- ✅ 建立可重用的依賴
- ✅ 依賴的層次結構
- ✅ 實際應用場景

---

## 🤔 什麼是依賴注入？

**依賴注入 (Dependency Injection, DI)** 是一種設計模式，讓函式需要的「依賴」由外部提供，而非自己建立。

### 先理解「依賴」是什麼

在程式碼中，如果 A 需要 B 才能運作，我們說 A「依賴」B。

```python
def get_user(user_id: int):
    db = Database()        # ← 這個端點「依賴」資料庫
    user = verify_token()  # ← 這個端點「依賴」認證結果
    return db.get(user_id)
```

### 生活化比喻

<div style="display: grid; grid-template-columns: 1fr 1fr; gap: 16px; margin: 16px 0;">
  <div style="border: 2px solid currentColor; border-radius: 8px; padding: 12px;">
    <div style="font-weight: bold; margin-bottom: 8px;">❌ 沒有依賴注入</div>
    <div>🍳 廚師自己要去買菜、洗菜、準備食材，然後才能做菜</div>
    <div style="margin-top: 8px; font-size: 0.9em;">→ 廚師負責太多事情，職責不清</div>
  </div>
  <div style="border: 2px solid currentColor; border-radius: 8px; padding: 12px;">
    <div style="font-weight: bold; margin-bottom: 8px;">✅ 有依賴注入</div>
    <div>🍳 廚師只管做菜，食材由助手準備好送到手邊</div>
    <div style="margin-top: 8px; font-size: 0.9em;">→ 各司其職，分工明確</div>
  </div>
</div>

### 依賴注入解決的三大問題

| 問題 | 沒有 DI | 有 DI |
|------|---------|-------|
| **重複程式碼** | 每個端點都要寫一樣的初始化邏輯 | 定義一次，到處使用 |
| **難以測試** | 無法輕易替換真實資料庫為測試用假資料 | 輕鬆替換依賴 |
| **緊耦合** | 函式直接建立依賴物件，改不動 | 依賴從外部傳入，彈性高 |

In [1]:
# ❌ 沒有依賴注入：每個函式都要重複建立相同的東西

def get_user_v1(user_id: int):
    # 每次都要建立資料庫連線
    db = create_database_connection()
    # 每次都要驗證 token
    token = get_token_from_header()
    current_user = verify_token(token)
    # 實際邏輯
    return db.get_user(user_id)

def get_orders_v1(user_id: int):
    # 又要重複建立資料庫連線
    db = create_database_connection()
    # 又要重複驗證 token
    token = get_token_from_header()
    current_user = verify_token(token)
    # 實際邏輯
    return db.get_orders(user_id)

print("❌ 問題：重複的程式碼太多！")

❌ 問題：重複的程式碼太多！


In [2]:
# ✅ 使用依賴注入：依賴由框架自動提供

from fastapi import Depends

# 定義依賴（只寫一次）
def get_db():
    return "Database Connection"

def get_current_user():
    return {"id": 1, "name": "小明"}

# 使用依賴（自動注入）
def get_user_v2(user_id: int, db = Depends(get_db), user = Depends(get_current_user)):
    # db 和 user 由 FastAPI 自動提供
    return {"user_id": user_id, "db": db, "current_user": user}

def get_orders_v2(user_id: int, db = Depends(get_db), user = Depends(get_current_user)):
    # 同樣的依賴，自動注入
    return {"user_id": user_id, "db": db, "current_user": user}

print("✅ 優點：依賴集中定義，重複使用！")

✅ 優點：依賴集中定義，重複使用！


---

## 🔧 基本用法

使用 `Depends()` 來聲明依賴。FastAPI 會自動呼叫依賴函式並傳入結果。

### Depends 的運作流程

```
1. 請求進來
       ↓
2. FastAPI 看到 Depends(common_parameters)
       ↓
3. FastAPI 呼叫 common_parameters() 函式
       ↓
4. 把回傳值傳給你的端點函式
       ↓
5. 你的端點函式執行
```

### 語法解析

```python
@app.get("/items")
def get_items(commons: dict = Depends(common_parameters)):
    #           ↑              ↑
    #     參數名稱          Depends(依賴函式)
    # commons 會接收 common_parameters() 的回傳值
```

### 為什麼要用 Depends？

直接呼叫函式不行嗎？

```python
# ❌ 直接呼叫 - 無法自動處理請求參數
def get_items():
    commons = common_parameters()  # 參數從哪來？

# ✅ Depends - FastAPI 自動從請求解析參數傳給依賴函式
def get_items(commons: dict = Depends(common_parameters)):
    pass
```

In [3]:
from fastapi import FastAPI, Depends, Query
from typing import Optional

app = FastAPI()

# 定義一個依賴函式
def common_parameters(
    q: Optional[str] = None,
    skip: int = Query(default=0, ge=0),
    limit: int = Query(default=10, ge=1, le=100)
):
    """共用的查詢參數"""
    return {"q": q, "skip": skip, "limit": limit}

# 在多個端點中使用同一個依賴
@app.get("/items")
def get_items(commons: dict = Depends(common_parameters)):
    """取得項目列表"""
    return {"message": "取得項目", **commons}

@app.get("/users")
def get_users(commons: dict = Depends(common_parameters)):
    """取得用戶列表"""
    return {"message": "取得用戶", **commons}

In [4]:
# 測試
from fastapi.testclient import TestClient

client = TestClient(app)

print("🧪 測試共用參數依賴\n")

# 測試 /items
response = client.get("/items?q=search&skip=5&limit=20")
print(f"GET /items: {response.json()}")

# 測試 /users (使用預設值)
response = client.get("/users")
print(f"GET /users: {response.json()}")

🧪 測試共用參數依賴

GET /items: {'message': '取得項目', 'q': 'search', 'skip': 5, 'limit': 20}
GET /users: {'message': '取得用戶', 'q': None, 'skip': 0, 'limit': 10}


---

## 🏗️ 類別作為依賴

可以使用類別來組織依賴，更加結構化。

In [5]:
from fastapi import FastAPI, Depends, Query
from typing import Optional

app = FastAPI()

class CommonQueryParams:
    """共用查詢參數類別"""
    
    def __init__(
        self,
        q: Optional[str] = None,
        skip: int = Query(default=0, ge=0),
        limit: int = Query(default=10, ge=1, le=100)
    ):
        self.q = q
        self.skip = skip
        self.limit = limit

@app.get("/items")
def get_items(commons: CommonQueryParams = Depends()):
    """使用類別作為依賴
    
    Depends() 沒有參數時，會自動使用類型標註的類別
    """
    return {
        "q": commons.q,
        "skip": commons.skip,
        "limit": commons.limit
    }

# 測試
client = TestClient(app)
response = client.get("/items?q=test&skip=10")
print(f"類別依賴: {response.json()}")

類別依賴: {'q': 'test', 'skip': 10, 'limit': 10}


---

## 🔗 依賴的依賴（巢狀依賴）

依賴可以有自己的依賴，形成**依賴鏈**。這是 FastAPI 依賴系統最強大的特性之一。

### 真實場景：權限檢查

```
要檢查是不是管理員...
  → 首先要知道是誰（取得當前用戶）
    → 首先要有合法的 Token
      → 首先要從 Header 取得 Token
```

這就是一個依賴鏈！

### 依賴鏈的好處

| 好處 | 說明 |
|------|------|
| **關注點分離** | 每個依賴只做一件事 |
| **重複使用** | `get_current_user` 可以在多處使用 |
| **容易測試** | 可以單獨測試每一層 |
| **靈活組合** | 需要用戶資訊？加 `get_current_user`。需要管理員？加 `get_admin_user` |

### 實際建構依賴鏈

下面我們來建立一個三層的認證依賴鏈：

In [6]:
from fastapi import FastAPI, Depends, HTTPException, Header
from typing import Optional

app = FastAPI()

# 第一層依賴：取得 Token
def get_token(authorization: Optional[str] = Header(default=None)):
    """從 Header 取得 Token"""
    if not authorization:
        raise HTTPException(status_code=401, detail="缺少認證 Token")
    # 假設格式是 "Bearer <token>"
    if not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Token 格式錯誤")
    return authorization[7:]  # 移除 "Bearer "

# 第二層依賴：驗證 Token 並取得用戶
def get_current_user(token: str = Depends(get_token)):
    """依賴於 get_token"""
    # 模擬驗證 Token
    if token == "valid_token":
        return {"id": 1, "name": "小明", "role": "user"}
    elif token == "admin_token":
        return {"id": 0, "name": "管理員", "role": "admin"}
    raise HTTPException(status_code=401, detail="Token 無效")

# 第三層依賴：檢查是否為管理員
def get_admin_user(user: dict = Depends(get_current_user)):
    """依賴於 get_current_user"""
    if user["role"] != "admin":
        raise HTTPException(status_code=403, detail="需要管理員權限")
    return user

@app.get("/users/me")
def get_me(user: dict = Depends(get_current_user)):
    """取得當前用戶（需登入）"""
    return user

@app.get("/admin/dashboard")
def admin_dashboard(admin: dict = Depends(get_admin_user)):
    """管理員儀表板（需管理員權限）"""
    return {"message": f"歡迎，{admin['name']}！", "data": "機密資料"}

In [7]:
# 測試依賴鏈
client = TestClient(app)

print("🔗 依賴鏈測試\n")

# 沒有 Token
print("1. 沒有 Token:")
response = client.get("/users/me")
print(f"   狀態: {response.status_code}, 訊息: {response.json()['detail']}")

# 有效 Token (一般用戶)
print("\n2. 一般用戶:")
response = client.get("/users/me", headers={"Authorization": "Bearer valid_token"})
print(f"   {response.json()}")

# 一般用戶嘗試訪問管理員頁面
print("\n3. 一般用戶訪問管理員頁面:")
response = client.get("/admin/dashboard", headers={"Authorization": "Bearer valid_token"})
print(f"   狀態: {response.status_code}, 訊息: {response.json()['detail']}")

# 管理員訪問
print("\n4. 管理員訪問:")
response = client.get("/admin/dashboard", headers={"Authorization": "Bearer admin_token"})
print(f"   {response.json()}")

🔗 依賴鏈測試

1. 沒有 Token:
   狀態: 401, 訊息: 缺少認證 Token

2. 一般用戶:
   {'id': 1, 'name': '小明', 'role': 'user'}

3. 一般用戶訪問管理員頁面:
   狀態: 403, 訊息: 需要管理員權限

4. 管理員訪問:
   {'message': '歡迎，管理員！', 'data': '機密資料'}


### 依賴鏈圖解

<div style="border: 2px solid currentColor; border-radius: 8px; padding: 16px; margin: 16px 0;">
  <div style="text-align: center;">
    <div style="display: inline-block; border: 1px solid currentColor; padding: 8px 16px; border-radius: 4px;">API 端點</div>
    <div style="margin: 8px 0;">⬇️ Depends</div>
    <div style="display: inline-block; border: 1px solid currentColor; padding: 8px 16px; border-radius: 4px;">get_admin_user</div>
    <div style="margin: 8px 0;">⬇️ Depends</div>
    <div style="display: inline-block; border: 1px solid currentColor; padding: 8px 16px; border-radius: 4px;">get_current_user</div>
    <div style="margin: 8px 0;">⬇️ Depends</div>
    <div style="display: inline-block; border: 1px solid currentColor; padding: 8px 16px; border-radius: 4px;">get_token</div>
    <div style="margin: 8px 0;">⬇️ Header</div>
    <div style="display: inline-block; border: 1px solid currentColor; padding: 8px 16px; border-radius: 4px;">Authorization Header</div>
  </div>
</div>

---

## 🗃️ 資料庫連線依賴（yield 依賴）

依賴注入最常見也最重要的用途：**管理需要清理的資源**。

### 問題：資源洩漏

```python
# ❌ 這樣寫會有問題
@app.get("/data")
def get_data():
    db = Database()
    db.connect()
    result = db.query("SELECT * FROM users")
    # 如果這裡出錯，db.disconnect() 不會被呼叫！
    db.disconnect()
    return result
```

### 解決方案：yield 依賴

`yield` 依賴可以確保：
1. 請求開始時：執行 `yield` 之前的程式碼（取得資源）
2. 請求結束後：執行 `yield` 之後的程式碼（釋放資源）
3. **即使出錯也會執行清理！**（因為用 `try/finally`）

### yield 依賴的結構

```python
def get_db():
    db = Database()     # 1️⃣ 建立資源
    db.connect()        # 2️⃣ 初始化
    try:
        yield db        # 3️⃣ 提供給端點使用
    finally:
        db.disconnect() # 4️⃣ 請求結束後清理（一定會執行）
```

### 執行時序圖

```
請求進來
    ↓
執行 yield 之前的程式碼 (建立連線)
    ↓
yield db → 端點函式執行
    ↓
端點函式回傳
    ↓
執行 yield 之後的程式碼 (關閉連線)
    ↓
回應發送給客戶端
```

In [8]:
from fastapi import FastAPI, Depends
from typing import Generator

app = FastAPI()

# 模擬資料庫連線類別
class Database:
    def __init__(self):
        self.connected = False
    
    def connect(self):
        self.connected = True
        print("📡 資料庫已連線")
    
    def disconnect(self):
        self.connected = False
        print("🔌 資料庫已斷線")
    
    def query(self, sql: str):
        if not self.connected:
            raise Exception("未連線")
        return f"執行: {sql}"

# 使用 yield 的依賴（確保資源釋放）
def get_db() -> Generator[Database, None, None]:
    """取得資料庫連線
    
    使用 yield 確保請求結束後會執行清理程式碼
    """
    db = Database()
    db.connect()
    try:
        yield db  # 提供給端點使用
    finally:
        db.disconnect()  # 請求結束後執行

@app.get("/data")
def get_data(db: Database = Depends(get_db)):
    result = db.query("SELECT * FROM users")
    return {"result": result}

# 測試
client = TestClient(app)
print("🧪 測試資料庫依賴：\n")
response = client.get("/data")
print(f"\n回應: {response.json()}")

🧪 測試資料庫依賴：

📡 資料庫已連線
🔌 資料庫已斷線

回應: {'result': '執行: SELECT * FROM users'}


### yield 依賴的執行順序

```python
def get_db():
    db = Database()      # 1️⃣ 請求開始時執行
    db.connect()         # 2️⃣ 連線資料庫
    try:
        yield db         # 3️⃣ 提供給端點使用
    finally:
        db.disconnect()  # 4️⃣ 請求結束後清理
```

---

## 🎛️ 全域依賴

可以在應用層級設定依賴，自動套用到所有端點。

In [9]:
from fastapi import FastAPI, Depends, Header, HTTPException
from typing import Optional

# 全域依賴：API Key 驗證
def verify_api_key(x_api_key: Optional[str] = Header(default=None)):
    if x_api_key != "secret-api-key":
        raise HTTPException(status_code=403, detail="無效的 API Key")
    return x_api_key

# 將依賴套用到整個應用
app = FastAPI(dependencies=[Depends(verify_api_key)])

@app.get("/public")  # 也需要 API Key
def public_endpoint():
    return {"message": "公開端點"}

@app.get("/private")  # 也需要 API Key
def private_endpoint():
    return {"message": "私有端點"}

# 測試
client = TestClient(app)

print("🔑 全域依賴測試\n")

# 沒有 API Key
response = client.get("/public")
print(f"沒有 API Key: {response.status_code}")

# 有 API Key
response = client.get("/public", headers={"X-API-Key": "secret-api-key"})
print(f"有 API Key: {response.json()}")

🔑 全域依賴測試

沒有 API Key: 403
有 API Key: {'message': '公開端點'}


---

## 📦 實際應用範例

綜合運用依賴注入建立一個完整的 API。

In [10]:
from fastapi import FastAPI, Depends, HTTPException, Header
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

app = FastAPI()

# ===== 模型 =====
class User(BaseModel):
    id: int
    name: str
    role: str

class Item(BaseModel):
    id: int
    name: str
    owner_id: int

# ===== 模擬資料 =====
USERS = {
    "token_user1": User(id=1, name="小明", role="user"),
    "token_admin": User(id=0, name="管理員", role="admin"),
}

ITEMS = [
    Item(id=1, name="物品A", owner_id=1),
    Item(id=2, name="物品B", owner_id=1),
    Item(id=3, name="物品C", owner_id=2),
]

# ===== 依賴 =====

def get_current_user(
    authorization: Optional[str] = Header(default=None)
) -> User:
    """取得當前登入用戶"""
    if not authorization or not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="請先登入")
    
    token = authorization[7:]
    user = USERS.get(token)
    if not user:
        raise HTTPException(status_code=401, detail="Token 無效")
    return user

def require_admin(user: User = Depends(get_current_user)) -> User:
    """需要管理員權限"""
    if user.role != "admin":
        raise HTTPException(status_code=403, detail="需要管理員權限")
    return user

class Pagination:
    """分頁依賴"""
    def __init__(
        self,
        page: int = 1,
        size: int = 10
    ):
        self.page = max(1, page)
        self.size = min(100, max(1, size))
        self.skip = (self.page - 1) * self.size

# ===== API 端點 =====

@app.get("/me")
def get_me(user: User = Depends(get_current_user)):
    """取得當前用戶資訊"""
    return user

@app.get("/my-items")
def get_my_items(
    user: User = Depends(get_current_user),
    pagination: Pagination = Depends()
):
    """取得當前用戶的物品"""
    my_items = [item for item in ITEMS if item.owner_id == user.id]
    return {
        "user": user.name,
        "items": my_items[pagination.skip:pagination.skip + pagination.size],
        "page": pagination.page,
        "total": len(my_items)
    }

@app.get("/admin/all-items")
def get_all_items(
    admin: User = Depends(require_admin),
    pagination: Pagination = Depends()
):
    """管理員：取得所有物品"""
    return {
        "admin": admin.name,
        "items": ITEMS[pagination.skip:pagination.skip + pagination.size],
        "total": len(ITEMS)
    }

In [11]:
# 測試完整範例
client = TestClient(app)

print("📦 完整範例測試\n")

# 一般用戶
print("👤 一般用戶 (小明):")
headers = {"Authorization": "Bearer token_user1"}

response = client.get("/me", headers=headers)
print(f"  /me: {response.json()}")

response = client.get("/my-items", headers=headers)
print(f"  /my-items: {response.json()}")

response = client.get("/admin/all-items", headers=headers)
print(f"  /admin/all-items: {response.status_code} - {response.json()['detail']}")

# 管理員
print("\n👑 管理員:")
admin_headers = {"Authorization": "Bearer token_admin"}

response = client.get("/admin/all-items", headers=admin_headers)
print(f"  /admin/all-items: {response.json()}")

📦 完整範例測試

👤 一般用戶 (小明):
  /me: {'id': 1, 'name': '小明', 'role': 'user'}
  /my-items: {'user': '小明', 'items': [{'id': 1, 'name': '物品A', 'owner_id': 1}, {'id': 2, 'name': '物品B', 'owner_id': 1}], 'page': 1, 'total': 2}
  /admin/all-items: 403 - 需要管理員權限

👑 管理員:
  /admin/all-items: {'admin': '管理員', 'items': [{'id': 1, 'name': '物品A', 'owner_id': 1}, {'id': 2, 'name': '物品B', 'owner_id': 1}, {'id': 3, 'name': '物品C', 'owner_id': 2}], 'total': 3}


---

## 📋 依賴注入速查表

| 用法 | 說明 |
|------|------|
| `Depends(func)` | 使用函式作為依賴 |
| `Depends(Class)` | 使用類別作為依賴 |
| `Depends()` | 自動使用類型標註的類別 |
| `yield` 依賴 | 確保資源釋放 |
| 全域依賴 | `FastAPI(dependencies=[...])` |

---

## 🎯 練習題

建立一個「請求日誌」依賴，記錄每次請求的資訊。

In [ ]:
# 練習：建立請求日誌依賴

# 需求：
# 1. 記錄請求開始時間
# 2. 請求結束後印出執行時間
# 3. 使用 yield 依賴

# TODO: 在這裡寫你的程式碼
# def log_request():
#     pass

<details>
<summary>📝 點擊查看解答</summary>

```python
from fastapi import FastAPI, Depends, Request
from datetime import datetime
from typing import Generator

def log_request() -> Generator[datetime, None, None]:
    start_time = datetime.now()
    print(f"📥 請求開始: {start_time}")
    try:
        yield start_time
    finally:
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        print(f"📤 請求結束: 耗時 {duration:.3f} 秒")

app = FastAPI()

@app.get("/test")
def test_endpoint(start: datetime = Depends(log_request)):
    return {"started_at": start.isoformat()}
```

</details>

---

## 📋 本節重點

| 概念 | 說明 |
|------|------|
| 依賴注入 | 讓函式需要的依賴由外部提供 |
| `Depends()` | 聲明依賴的方式 |
| 依賴鏈 | 依賴可以有自己的依賴 |
| `yield` 依賴 | 確保請求結束後釋放資源 |
| 全域依賴 | 套用到所有端點 |

---

## 📋 第二章總結

恭喜你完成了第二章！讓我們回顧：

| 單元 | 學習內容 |
|------|----------|
| 2.1 | Python 類型提示 |
| 2.2 | Pydantic 資料驗證 |
| 2.3 | 依賴注入系統 |

---

## ⏭️ 下一步

👉 **[第三章：路由設計](../03_routing/)** - 學習路由組織與 APIRouter